In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os, platform
system = platform.platform()
system = system[0:system.index('-')]
print("Your operating system:", system)

if system == "Linux":
    dataDir = os.path.join('/kaggle/input', os.listdir('/kaggle/input')[0] + "/")
    print(os.listdir(dataDir))
elif system == "macOS":
    dataDir = 'Data/'
    print(os.listdir(dataDir))
else:
    raise Exception("Please manually set the `dataDir` variable to the directory where the data files are located")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Your operating system: Linux
['sample_submission.csv', 'train-metadata.csv', 'test-metadata.csv', 'test-image.hdf5', 'train-image', 'train-image.hdf5']


In [2]:
train_gt = pd.read_csv(dataDir + "train-metadata.csv")[['isic_id', 'target']].to_numpy()
train_size = len(train_gt)
print("\nTraining Set Size:", train_size)

unique, unique_counts = np.unique(train_gt[:, 1], return_counts=True)
for element, element_count in zip(unique, unique_counts):
    if element == 0:
        benign = element_count / train_size
        print(f"% of Benign Cases: {benign*100}%")
    elif element == 1:
        malignant = element_count / train_size
        print(f"% of Malignant Cases: {malignant*100}%")

/tmp/ipykernel_33/3368939254.py:1: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  train_gt = pd.read_csv(dataDir + "train-metadata.csv")[['isic_id', 'target']].to_numpy()



Training Set Size: 401059
% of Benign Cases: 99.90200943003398%
% of Malignant Cases: 0.09799056996601498%


In [3]:
# Naive Approach
submission = pd.read_csv(dataDir + "sample_submission.csv")
submission['target'] = 1 - malignant

if system == "Linux":
    dataDir = "/kaggle/working/"

submission.to_csv(dataDir + "submission.csv", index=None)
submission

"""
import h5py
test = dataDir + "test-image.hdf5"

with h5py.File(test, "r") as f:
    keys = list(f.keys())
"""

'\nimport h5py\ntest = dataDir + "test-image.hdf5"\n\nwith h5py.File(test, "r") as f:\n    keys = list(f.keys())\n'